***CONECTANDO AO MONGO DB***

In [2]:
from pymongo.mongo_client import MongoClient

In [3]:
client = MongoClient()

try:
    client.admin.command('ping')
    print('conectou!!')
except Exception as e:
    print(e)

conectou!!


***CRIANDO BANCO E COLEÇÃO***

In [4]:
db = client["db_produtos_amazon"]

In [5]:
collection = db["produtos_amazon"]

In [6]:
teste = {
    "teste" : 1
}

collection.insert_one(teste)

InsertOneResult(ObjectId('67f5de67e969d7f66c738d12'), acknowledged=True)

In [7]:
client.list_database_names()

['admin',
 'config',
 'db_avaliacao_credito',
 'db_cotacoes',
 'db_dados_eleitorais',
 'db_eleicao_pe',
 'db_pipeline_abandono_escolar',
 'db_produtos',
 'db_produtos_amazon',
 'db_reclamacoes',
 'local']

***COMEÇANDO A USAR O SPARK***

In [8]:
import os
from pyspark.sql import SparkSession

In [9]:
os.environ["SPARK_HOME"] = "/opt/spark/spark-3.5.5-bin-hadoop3"

In [10]:
import findspark

In [11]:
findspark.init()

In [12]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

25/04/08 23:42:11 WARN Utils: Your hostname, dg-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlp1s0)
25/04/08 23:42:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/08 23:42:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
spark

***lENDO OS DADOS***

In [14]:
path_csv = "../data_raw/amz_br_total_products_data_processed.csv"

In [88]:
data_product_amazon = spark.read.csv(
    path_csv,
    sep=',',
    header=True,
    inferSchema=True,
    quote='"',
    escape='"',
    
)


In [89]:
data_product_amazon.count()

1344513

In [90]:
data_product_amazon.limit(5).show(truncate=False)

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------+---------------------------------------+-----+-------+-------+---------+--------------------------+------------+-----------------+
|asin      |title                                                                                                                                                                                                |imgUrl                                                        |productURL                             |stars|reviews|price  |listPrice|categoryName              |isBestSeller|boughtInLastMonth|
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [91]:
data_product_amazon.columns

['asin',
 'title',
 'imgUrl',
 'productURL',
 'stars',
 'reviews',
 'price',
 'listPrice',
 'categoryName',
 'isBestSeller',
 'boughtInLastMonth']

In [92]:
data_product_amazon.createOrReplaceTempView("produtos_view")

In [111]:
data_product_amazon = spark.sql("SELECT title,stars,reviews,price,listPrice,categoryName FROM produtos_view")

In [112]:
spark.sql("SELECT DISTINCT title FROM  produtos_view").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Ração Úmida Para Gatos Whiskas Sachê Frango ao Molho Adultos 85g                                                                                         

In [113]:
spark.sql("SELECT DISTINCT stars FROM  produtos_view  order by stars").show()

+-----+
|stars|
+-----+
|  0.0|
|  1.0|
|  1.1|
|  1.2|
|  1.3|
|  1.4|
|  1.5|
|  1.6|
|  1.7|
|  1.8|
|  1.9|
|  2.0|
|  2.1|
|  2.2|
|  2.3|
|  2.4|
|  2.5|
|  2.6|
|  2.7|
|  2.8|
+-----+
only showing top 20 rows



In [114]:
data_product_amazon.printSchema()

root
 |-- title: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- listPrice: double (nullable = true)
 |-- categoryName: string (nullable = true)



***TRANSFORMANDO PARA JSON***

In [115]:
data_product_amazon.write.json(path="../data_processed/data_json",mode='overwrite')